<a href="https://colab.research.google.com/github/chenetheophile/ClassificationPanneaux/blob/main/ClassificationPanneauxSignalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import keras
from keras import utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


In [31]:
from google.colab import drive
from zipfile import ZipFile
import os
from PIL import Image
drive.mount('/content/drive')
ziplist=['./drive/MyDrive/Classification/GTSRB_Final_Test_Images.zip','./drive/MyDrive/Classification/GTSRB_Final_Training_Images.zip']
for zip in ziplist:
  extrait=ZipFile(zip, "r")
  extrait.extractall('./Classification')
  extrait.close()
def searchImage():
 for root, dirs, files in os.walk("./Classification"):
    path = root.split(os.sep)
    for file in files:
      if ".ppm" in file:
        Image.open(root+'/'+file).save(root+'/'+file[:-3]+"jpeg")
        os.remove(root+'/'+file)

searchImage()

None


In [32]:
from tensorflow import keras

train_ds = keras.utils.image_dataset_from_directory(
    directory='Classification/GTSRB/Final_Training/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
validation_ds = keras.utils.image_dataset_from_directory(
    directory='Classification/GTSRB/Final_Test/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

Found 39209 files belonging to 1 classes.
Found 12630 files belonging to 1 classes.


In [52]:
import csv
import matplotlib.pyplot as plt
import os
def readTrafficSign(path):
  images=[]
  labels=[]
  for i in range(len(os.listdir(path))):
    prefix=(path+'/'+format(i,'05d'))+'/'
    getfile=open(prefix+'GT-'+format(i,'05d')+'.csv')
    gtReader=csv.reader(getfile,delimiter=";")
    gtReader.__next__()
    for row in gtReader:
      images.append(plt.imread(prefix+row[0][:-3]+'jpeg'))
      labels.append(row[7])
    getfile.close()
  return images,labels

img,label=readTrafficSign('./Classification/GTSRB/Final_Training/Images')

In [53]:
print(len(img))

39209
